# Análise de dados dos jogos do Flamengo 2024

In [ ]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd 
import csv
from datetime import datetime

In [ ]:
# Importando os arquivos
jogos = pd.read_csv('../data/processed/matches_FLA2025_processed.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/processed/goal_scorers_2025_processed.csv')

## Estatísticas 

In [ ]:
# Calculando os gols pró, sofridos e saldo de gols
gols_pro = jogos.gols_casa.sum() + jogos.gols_fora.sum()
gols_contra = jogos.gols_sofridos_casa.sum() + jogos.gols_sofridos_fora.sum()
saldo_gols = gols_pro - gols_contra

In [ ]:
# Obtendo o número de vitórias, derrotas, empates e total de jogos
vitorias = jogos['resultado'].value_counts().iloc[0]
derrotas = jogos['resultado'].value_counts().iloc[2]
empates = jogos['resultado'].value_counts().iloc[1]
total_jogos = vitorias + empates + derrotas

# Gols pro e sofridos por jogo
gols_per_jogo = round(gols_pro/total_jogos,2)
gols_sofrido_jogo = round(gols_contra/total_jogos,2)

# Calculando o aproveitamento
aproveitamento = round((3*vitorias + 1*empates + 0*derrotas)/(3*total_jogos)*100,2)
aproveitamento_pontos = (3*vitorias + 1*empates + 0*derrotas)

In [ ]:
jogos

In [ ]:
jogos_marcou = jogos.marcou_gol.value_counts().iloc[0]
jogos_nao_marcou = jogos.marcou_gol.value_counts().iloc[1]

jogos_sofreu = jogos.sofreu_gol.value_counts(ascending=True).iloc[0]
jogos_nao_sofreu = jogos.sofreu_gol.value_counts(ascending=True).iloc[1]

In [ ]:
stats = pd.DataFrame(jogos[['id_match', 'marcou_gol', 'sofreu_gol', 'kick-off', 'resultado', 'locale', 'jogos_cumulativos']])

stats['gols_marcados'] = jogos.gols_casa + jogos.gols_fora
stats['gols_sofridos'] = jogos.gols_sofridos_casa + jogos.gols_sofridos_fora

stats['gols_marcados_acumulados'] = stats['gols_marcados'].cumsum()
stats['gols_sofridos_acumulados'] = stats['gols_sofridos'].cumsum()
stats['diferença_gols_acumulados'] = stats['gols_marcados_acumulados'] - stats['gols_sofridos_acumulados']

In [ ]:
print(
"""
Jogos: {0}
Vitórias: {1}
Empates: {2}
Derrotas: {3}*
Gols pro: {4}
Gols contra: {5}
Saldo de gols: +{6}
Gols/jogo: {7}
Gols sofridos/jogo: {8}
Jogos em que marcou: {9}
Jogos em que sofreu gol: {10} 
Aproveitamento: {11}%
Aproveitamento pontos: {12}/{13}

*Filipe Luis tem apenas {14} derrota no ano.
""".format(total_jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_gols,gols_per_jogo,
           gols_sofrido_jogo,jogos_marcou,jogos_sofreu,aproveitamento,aproveitamento_pontos, 3*total_jogos,derrotas-2))

### Criando tabelas com o número de gols e assistências

In [ ]:
# Fazendo o merge dos dataframes jogos e gols
df = pd.merge(jogos, gols, on = 'id_match')
df.head()

In [ ]:
# Quantidade de gols por jogador

df_gols_counts = pd.DataFrame(gols.gols.value_counts())
df_gols_counts = df_gols_counts.reset_index()
df_gols_counts.columns = ['Jogador', 'Gols'] 

# Quantidade de assistência por jogador

df_assist_counts = pd.DataFrame(gols.assist.value_counts())
df_assist_counts = df_assist_counts.reset_index()
df_assist_counts.columns = ['Jogador', 'Assistências'] 

new_df = pd.concat([df_assist_counts, df_gols_counts], axis=1)#.dropna(how='all')
print(new_df.to_markdown(index=False))

In [ ]:
# Unificando as tabelas gols e assistências
uni_df = pd.merge(df_gols_counts, df_assist_counts, on = 'Jogador', how ="left").fillna(0).head(15)

print(uni_df.to_markdown(index=False))

In [ ]:
stats['year'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%Y"))
stats['month'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%b"))
stats['weekday'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%a"))

In [ ]:
stats.to_csv('../data/processed/stats_FLA2025_processed.csv', index=False)

In [ ]:
stats